# [1581. Customer Who Visited but Did Not Make Any Transactions](https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transactions/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Visits

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| visit_id    | int     |
| customer_id | int     |
+-------------+---------+</pre>
visit_id is the column with unique values for this table.
This table contains information about the customers who visited the mall.
 

Table: Transactions

<pre>+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| transaction_id | int     |
| visit_id       | int     |
| amount         | int     |
+----------------+---------+</pre>
transaction_id is column with unique values for this table.
This table contains information about the transactions made during the visit_id.
 

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

Return the result table sorted in any order.

The result format is in the following example.

 

Example 1:

Input: 
Visits
<pre>+----------+-------------+
| visit_id | customer_id |
+----------+-------------+
| 1        | 23          |
| 2        | 9           |
| 4        | 30          |
| 5        | 54          |
| 6        | 96          |
| 7        | 54          |
| 8        | 54          |
+----------+-------------+</pre>
Transactions
<pre>+----------------+----------+--------+
| transaction_id | visit_id | amount |
+----------------+----------+--------+
| 2              | 5        | 310    |
| 3              | 5        | 300    |
| 9              | 5        | 200    |
| 12             | 1        | 910    |
| 13             | 2        | 970    |
+----------------+----------+--------+</pre>
Output: 
<pre>+-------------+----------------+
| customer_id | count_no_trans |
+-------------+----------------+
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |
+-------------+----------------+</pre>
Explanation: 
Customer with id = 23 visited the mall once and made one transaction during the visit with id = 12.
Customer with id = 9 visited the mall once and made one transaction during the visit with id = 13.
Customer with id = 30 visited the mall once and did not make any transactions.
Customer with id = 54 visited the mall three times. During 2 visits they did not make any transactions, and during one visit they made 3 transactions.
Customer with id = 96 visited the mall once and did not make any transactions.
As we can see, users with IDs 30 and 96 visited the mall one time without making any transactions. Also, user 54 visited the mall twice and did not make any transactions.

In [1]:
#pandas schema

import pandas as pd

data = [[1, 23], [2, 9], [4, 30], [5, 54], [6, 96], [7, 54], [8, 54]]
visits = pd.DataFrame(data, columns=['visit_id', 'customer_id']).astype({'visit_id':'Int64', 'customer_id':'Int64'})
data = [[2, 5, 310], [3, 5, 300], [9, 5, 200], [12, 1, 910], [13, 2, 970]]
transactions = pd.DataFrame(data, columns=['transaction_id', 'visit_id', 'amount']).astype({'transaction_id':'Int64', 'visit_id':'Int64', 'amount':'Int64'})


In [2]:
# to pyspark schema
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

visits_df = spark.createDataFrame(visits)
visits_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:02:13 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:02:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:02:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------+-----------+
|visit_id|customer_id|
+--------+-----------+
|1       |23         |
|2       |9          |
|4       |30         |
|5       |54         |
|6       |96         |
|7       |54         |
|8       |54         |
+--------+-----------+


In [3]:
transactions_df = spark.createDataFrame(transactions)
transactions_df.show(truncate=False)

+--------------+--------+------+
|transaction_id|visit_id|amount|
+--------------+--------+------+
|2             |5       |310   |
|3             |5       |300   |
|9             |5       |200   |
|12            |1       |910   |
|13            |2       |970   |
+--------------+--------+------+


In [4]:
# In spark Dataframe
import pyspark.sql.functions as f

visits_df\
    .join(transactions_df,on=['visit_id'],how='left')\
    .filter(f.col('transaction_id').isNull())\
    .groupBy('customer_id')\
    .agg(f.count('customer_id').alias('count_no_trans'))\
    .show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         54|             2|
|         96|             1|
|         30|             1|
+-----------+--------------+


In [5]:
# In spark SQL
visits_df.createOrReplaceTempView('visits')
transactions_df.createOrReplaceTempView('transactions')

spark.sql('''
select customer_id, count(customer_id) as count_no_trans
from visits 
left join transactions
on transactions.visit_id=visits.visit_id
where transaction_id is null
group by customer_id;
''').show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         54|             2|
|         96|             1|
|         30|             1|
+-----------+--------------+


In [6]:
spark.stop()